### Brief comments

- vendor37 was a very small contributor to revenue
- The closure on 1st April did see a fair amount of drop in revenue from the last week and from the avg of the last same weekdays but most of that was due to vendor26 seeing a drop
- The user base of vendor37 was almost eroded by around 92%
- While the closure did have a negative impact, it wasn't a large one by any means
- Overall revenue was declining on a monthly basis


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/DE_assignment')
print(os.getcwd())
import datetime as dt
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
pd.options.mode.chained_assignment = None
pd.options.display.float_format = '{:,}'.format

/content/drive/MyDrive/Colab Notebooks/DE_assignment


In [ ]:
df=pd.read_csv('m_mock_transaction.csv')

print('checking for null values')
print(df.isnull().sum())
print('\n')
print('checking data types')
print(df.dtypes)
print('\n')
print(len(df))
##Data is clean

total_rev=df['revenue'].sum()
total_user_cnt=df['account'].nunique()

###making fee assumptions


vm=pd.read_csv('m_vendor_map.csv')
vm.rename(columns={'vendor_id':'vendor'},inplace=True)
vm['fees_known']='Y'
vm.loc[(vm['fee_percentage'].isnull()),'fees_known']='N'


##joining vendor map to main df

df=pd.merge(df,vm,on=['vendor'],how='left')

vendor_user_cnt=df.groupby(['fees_known','vendor','fee_percentage'])['account'].nunique().reset_index().rename(columns={'account':'user_cnt'})

print(vendor_user_cnt)

vendor_rev=df.groupby(['fees_known','vendor'])['revenue'].sum().reset_index().rename(columns={'revenue':'total_rev'})

###Let us put 35 as assumed fees as it can be expected that smaller vendors will charge more fees. In any case they bring in lower revenue with the exception of vendor29
### Only vendor29 has been assumed to be 30 as it is larger than the rest of the unkwon vendors by a significant margin

df.loc[(df['fee_percentage'].isnull()),'fee_percentage']=35
df.loc[(df['vendor']=='nendor29'),'fee_percentage']=30

df['revenue_net_fees']=df['revenue']*(1-df['fee_percentage']/100)

#adding a dummy year column

df['year']=2020

##creating a date column
cols=["year","month","day"]
df['date'] = df[cols].apply(lambda x: '-'.join(x.values.astype(str)), axis=1)
df['date']=pd.to_datetime(df['date'],format='%Y-%m-%d')



checking for null values
account         0
sku             0
vendor          0
country_code    0
month           0
day             0
revenue         0
dtype: int64


checking data types
account          object
sku              object
vendor           object
country_code     object
month             int64
day               int64
revenue         float64
dtype: object


729200
  fees_known    vendor  fee_percentage  user_cnt
0          Y  vendor11             5.0     64803
1          Y  vendor26            20.0    252611
2          Y  vendor33            22.0     32858
3          Y  vendor36            25.0       769
4          Y  vendor37            30.0      1215
5          Y  vendor42            20.0     20355


### Part 1


-  Before closure – what did revenue look like for vendor37?
  - Proportion of total revenue
  - Revenue and transactions per day through vendor37
  - Regional usage details

In [ ]:

##Create a column for non vendor37

df['vendor37_flag']='other_vendors'
df.loc[(df['vendor']=='vendor37'),'vendor37_flag']='vendor_37'

### Filter only for the months vendor37 was active
df1=df[df['month']<4]


grp1=df1.groupby(['vendor37_flag'])['revenue'].sum().reset_index()
grp1['prop_rev_perc']=(grp1['revenue']/grp1['revenue'].sum())
grp1['prop_rev_perc']=grp1['prop_rev_perc'].round(4)
grp1['prop_rev_perc2']=(grp1['prop_rev_perc']*100).round(2)


print(grp1[['vendor37_flag','prop_rev_perc2']])
print('\n')
print('vendor37 only accounted for around 0.46% of revenue in the 3 months it was active')

fig = px.bar(grp1, x='vendor37_flag', y='prop_rev_perc',text_auto=True,title="Revenue share")
fig.layout.yaxis.tickformat = '0%'
print('\n')
fig.show()

grp2=df1.groupby(['vendor37_flag'])['revenue_net_fees'].sum().reset_index()
grp2['prop_rev_perc']=(grp2['revenue_net_fees']/grp2['revenue_net_fees'].sum())
grp2['prop_rev_perc']=grp2['prop_rev_perc'].round(4)
grp2['prop_rev_perc2']=(grp2['prop_rev_perc']*100).round(2)


print(grp2[['vendor37_flag','prop_rev_perc2']])

fig = px.bar(grp2, x='vendor37_flag', y='prop_rev_perc',text_auto=True,title="Revenue share")
fig.layout.yaxis.tickformat = '0%'
print('\n')
fig.show()

   vendor37_flag  prop_rev_perc2
0  other_vendors           99.54
1      vendor_37            0.46


vendor37 only accounted for around 0.46% of revenue in the 3 months it was active




   vendor37_flag  prop_rev_perc2
0  other_vendors           99.61
1      vendor_37            0.39




In [ ]:
###let's compare vendor37 to others during the time it was active

df1=df[df['month']<4]

vendor_rnking=df1.groupby(['vendor'])['revenue'].sum().round(2).reset_index().sort_values(['revenue'],ascending=False)

vendor_rnking['rnk']=vendor_rnking['revenue'].rank(ascending=False)

print(vendor_rnking)
print('\nEven though vendor37 was 5th in rank, the revenue it gave was just north of 10% of the 4th ranked vendor')

      vendor       revenue  rnk
4   vendor26 30,130,863.27  1.0
0   vendor11  8,659,461.06  2.0
7   vendor33  1,990,050.36  3.0
10  vendor42  1,791,567.51  4.0
9   vendor37    196,871.98  5.0
8   vendor36    158,289.36  6.0
6   vendor29    100,703.71  7.0
5   vendor27      1,556.67  8.0
3   vendor25        675.95  9.0
2   vendor20        330.22 10.0
1   vendor14        210.79 11.0

Even though vendor37 was 5th in rank, the revenue it gave was just north of 10% of the 4th ranked vendor


In [ ]:
###filtering only records for vendor37
df_v37=df[df['vendor']=='vendor37'].sort_values(['date'],ascending=True)

df_v37_daily_rev= df_v37.groupby(['date'])['revenue'].sum().reset_index().rename(columns={'revenue':'daily_rev'})
df_v37_daily_rev2= df_v37.groupby(['date'])['revenue_net_fees'].sum().reset_index().rename(columns={'revenue_net_fees':'daily_rev_net_fees'})


###Plotting a quick line chart

fig1 = px.line(df_v37_daily_rev, x="date", y="daily_rev",title='Daily revenue for vendor37')

fig1a = px.line(df_v37_daily_rev2, x="date", y="daily_rev_net_fees",title='Daily revenue for vendor37 net of fees')


##same for number of transactions

df_v37_daily_trans= df_v37.groupby(['date'])['sku'].count().reset_index().rename(columns={'sku':'tranactions_count'})

fig2 = px.line(df_v37_daily_trans, x="date", y="tranactions_count",title='Daily transactions for vendor37')


print('The avg daily revenue for vendor37 was:', df_v37_daily_rev['daily_rev'].mean().round(2))
print('\n')

print('The avg daily revenue net of fees for vendor37 was:', df_v37_daily_rev2['daily_rev_net_fees'].mean().round(2))
print('\n')

print('The avg daily transactions for vendor37 was:', df_v37_daily_trans['tranactions_count'].mean().round(2))
print('\n')
fig1.show()
print('\n')
fig1a.show()
print('\n')
fig2.show()

The avg daily revenue for vendor37 was: 2316.14


The avg daily revenue net of fees for vendor37 was: 1621.3


The avg daily transactions for vendor37 was: 22.11




In [ ]:
###Looking at regional usage for vendor37
print('\n')
print('It can be seen that the top 5 regions account for more that 80% of revenue - country66 and count45 leading \nthe shares by significant margins')

print('\n')
print('The distribution of users is very similar to the revenue shares')

df_v37_regions=df_v37.groupby(['country_code'])['revenue'].sum().reset_index().sort_values(['revenue'],ascending=False)
df_v37_regions['revenue_share']=df_v37_regions['revenue']/df_v37_regions['revenue'].sum()
df_v37_regions['cc_rank']=df_v37_regions['revenue'].rank(ascending=False)
df_v37_regions['top5']='Others'
df_v37_regions.loc[(df_v37_regions['cc_rank']) <= 5, 'top5']=df_v37_regions['country_code']

fig3=px.pie(df_v37_regions, values='revenue_share', names='top5',hover_data=['revenue'],
            title='Share of revenue by vendor37 across regions')
fig3.update_traces(textposition='inside', textinfo='percent+label')
fig3.update_layout(title_y=0.9)
print('\n')
fig3.show()

##looking at the user concentration

df_v37_regions_users=df_v37.groupby(['country_code'])['account'].nunique().reset_index().\
sort_values(['account'],ascending=False).rename(columns={'account':'user_count'})

df_v37_regions_users['user_share']=df_v37_regions_users['user_count']/df_v37_regions_users['user_count'].sum()
df_v37_regions_users['cc_rank']=df_v37_regions_users['user_count'].rank(ascending=False)
df_v37_regions_users['top5']='Others'
df_v37_regions_users.loc[(df_v37_regions_users['cc_rank']) <= 5, 'top5']=df_v37_regions_users['country_code']

fig4=px.pie(df_v37_regions_users, values='user_count', names='top5',hover_data=['user_count'],
            title='Distribution of user by vendor37 across regions')
fig4.update_traces(textposition='inside', textinfo='percent+label')
fig4.update_layout(title_y=0.9)
print('\n')
fig4.show()







It can be seen that the top 5 regions account for more that 80% of revenue - country66 and count45 leading 
the shares by significant margins


The distribution of users is very similar to the revenue shares




### What is an upper bound for revenue loss as a result of the closure?

In order to arrive at an estimate for this, the following steps are performed

- We calculate the month on month growth rates for the 3 months
- We calculate the compounded growth rate from the first period to the last using a modified CAGR formula
- We calculate the monthly std deviation using the following formula
- We apply the avg volatility calculated and apply it to the expected projected value at 1,3 and 3 times it to get three level of variance


In [ ]:
###Calculating daily changes
df_v37=df[df['vendor']=='vendor37'].sort_values(['date'],ascending=True)

df_v37_daily_rev= df_v37.groupby(['date'])['revenue'].sum().reset_index()\
.rename(columns={'revenue':'daily_rev'})

df_v37_daily_rev['daily_rev_lag']=df_v37_daily_rev['daily_rev'].shift(1)
df_v37_daily_rev['perc_chng']=(df_v37_daily_rev['daily_rev']/df_v37_daily_rev['daily_rev_lag'])-1

daily_stddev=np.std(df_v37_daily_rev['perc_chng'])
print(daily_stddev)
print('\n')

###Now we have a measure for the daily volatility of revenue

###Let us now extrapolate the monthly growth
###calculating month on month growth
df_v37_monthly_rev= df_v37.groupby(['month'])['revenue'].sum().reset_index()\
.rename(columns={'revenue':'monthly_rev'})

mnth_min=df_v37_monthly_rev['month'].min()
mnth_max=df_v37_monthly_rev['month'].max()

prd_diff=mnth_max-mnth_min

starting_rev=df_v37_monthly_rev.loc[df_v37_monthly_rev['month'] == mnth_min,
                                    'monthly_rev'].values[0]

ending_rev=df_v37_monthly_rev.loc[df_v37_monthly_rev['month'] == mnth_max,
                                  'monthly_rev'].values[0]

cmpgrth=((ending_rev/starting_rev)**(1/prd_diff))-1

###extrapolating and projecting the revenue for the upcoming months

for i in range(mnth_max+1,mnth_max+4):

  l2=[]
  monthly_rev_proj=ending_rev*(1+cmpgrth)

  ending_rev=monthly_rev_proj

  l2.append(i)
  l2.append(monthly_rev_proj)

  df_v37_monthly_rev.loc[len(df_v37_monthly_rev)]=l2

df_v37_monthly_rev['projection']='N'

df_v37_monthly_rev.loc[df_v37_monthly_rev['month']>3,'projection']='Y'
df_v37_monthly_rev

day_cnt=df.groupby(['month'])['day'].nunique().reset_index().\
rename(columns={'day':'day_count'})

df_v37_monthly_rev=pd.merge(df_v37_monthly_rev,day_cnt,on=['month'],how='inner')
df_v37_monthly_rev_proj=df_v37_monthly_rev[df_v37_monthly_rev['projection']=='Y']
df_v37_monthly_rev_proj['hist_vol']=daily_stddev
df_v37_monthly_rev_proj['sqrt_day_cnt']=(df_v37_monthly_rev_proj['day_count'])**0.5
df_v37_monthly_rev_proj['mnthly_std_dev_perc']=(df_v37_monthly_rev_proj['sqrt_day_cnt']*df_v37_monthly_rev_proj['hist_vol'])/100
df_v37_monthly_rev_proj['upper_bound_1sig']=df_v37_monthly_rev_proj['monthly_rev']*(1+df_v37_monthly_rev_proj['mnthly_std_dev_perc'])
df_v37_monthly_rev_proj['upper_bound_2sig']=df_v37_monthly_rev_proj['monthly_rev']*(1+df_v37_monthly_rev_proj['mnthly_std_dev_perc']*2)
df_v37_monthly_rev_proj['upper_bound_3sig']=df_v37_monthly_rev_proj['monthly_rev']*(1+df_v37_monthly_rev_proj['mnthly_std_dev_perc']*3)

upper_bound_df=df_v37_monthly_rev_proj[['month','monthly_rev','upper_bound_1sig','upper_bound_2sig','upper_bound_3sig']].rename(columns={'monthly_rev':'expected_rev'})

print('The total upper bound of revenue loss is '+ str(upper_bound_df['upper_bound_1sig'].sum().round(2)) + ' at the 1 sigma level')
print('The total upper bound of revenue loss is '+ str(upper_bound_df['upper_bound_2sig'].sum().round(2)) + ' at the 2 sigma level')
print('The total upper bound of revenue loss is '+ str(upper_bound_df['upper_bound_3sig'].sum().round(2)) + ' at the 3 sigma level')
print('\n')
print('\n')


fig3=go.Figure()

fig3.add_trace(
    go.Bar(
        x=df_v37_monthly_rev[df_v37_monthly_rev['projection']=='N']['month'],
        y=df_v37_monthly_rev[df_v37_monthly_rev['projection']=='N']['monthly_rev'],name="Actual rev",
        text=df_v37_monthly_rev[df_v37_monthly_rev['projection']=='N']['monthly_rev'].round(2)
    ))

fig3.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 1,
        
    )
)

fig3.update_yaxes(tickformat=",.0s")

fig3.add_trace(
    go.Bar(
        x=upper_bound_df['month'],
        y=upper_bound_df['expected_rev'],name='Projected rev',
        text=upper_bound_df['expected_rev'].round(2)
    ))

fig3.add_trace(
    go.Scatter(
        x=upper_bound_df['month'],
        y=upper_bound_df['upper_bound_1sig'],name='upper_bound_1sig',
        text=upper_bound_df['upper_bound_1sig'].round(2),
        mode='lines+markers'
    ))

fig3.add_trace(
    go.Scatter(
        x=upper_bound_df['month'],
        y=upper_bound_df['upper_bound_2sig'],name='upper_bound_2sig',
        text=upper_bound_df['upper_bound_2sig'].round(2),
        mode='lines+markers'
    ))

fig3.add_trace(
    go.Scatter(
        x=upper_bound_df['month'],
        y=upper_bound_df['upper_bound_3sig'],name='upper_bound_3sig',
        text=upper_bound_df['upper_bound_3sig'].round(2),
        mode='lines+markers'
    ))



fig3.update(layout_yaxis_range = [20000,90000])
fig3.show()


0.566324380193605


The total upper bound of revenue loss is 169230.52 at the 1 sigma level
The total upper bound of revenue loss is 174349.09 at the 2 sigma level
The total upper bound of revenue loss is 179467.67 at the 3 sigma level






### Number of users expected to be affected by the closure?

In order to answer this, we look the data inly till 31st of month 3.
This is done as we assume we have no idea of how users are going to behave after closure. This is a view from a point-in-time.

- We break up the users into 2 groups
  - Users who have only transacted with vendor37
  - Users who have transacted with vendor37

In [ ]:
df_users=df[df['month']<4]

df_users_v37=df_users[df_users['vendor']=='vendor37']

df_all_users_rev=df_users.groupby(['account'])['revenue'].sum().reset_index().rename(columns={'revenue':'total_user_revenue'})

df_v37_users_rev=df_users_v37.groupby(['account'])['revenue'].sum().reset_index().rename(columns={'revenue':'v37_rev'})

df_all_users_rev=pd.merge(df_all_users_rev,df_v37_users_rev,on=['account'],how='left')

df_all_users_rev.loc[(df_all_users_rev['v37_rev'].isnull()),'v37_rev']=0
                      
df_all_users_rev['v37_rev_share']=df_all_users_rev['v37_rev']/df_all_users_rev['total_user_revenue']

df_all_users_rev.sort_values(['v37_rev_share','account'],ascending=[False,True],inplace=True)

v37_user_type=df_all_users_rev[df_all_users_rev['v37_rev_share']>0]

v37_user_type['user_type']=np.nan

v37_user_type.loc[(v37_user_type['v37_rev_share']==1),'user_type']='exclusive user'

v37_user_type.loc[(v37_user_type['v37_rev_share']<1) & (v37_user_type['v37_rev_share']>=0.6),'user_type']='heavy user'

v37_user_type.loc[(v37_user_type['v37_rev_share']<0.6) & (v37_user_type['v37_rev_share']>=0.4),'user_type']='medium user'

v37_user_type.loc[(v37_user_type['v37_rev_share']<0.4) & (v37_user_type['v37_rev_share']>=0.1),'user_type']='low user'

v37_user_type.loc[(v37_user_type['v37_rev_share']<0.1),'user_type']='very low user'

v37_user_dist=v37_user_type.groupby(['user_type'])['account'].nunique().reset_index().rename(columns={'account':'user_count'})

print(v37_user_dist)
print('\n')
print('vendor37 had around 1039 exclusive users and 111 heavy users. They would be most likely to be affected')


        user_type  user_count
0  exclusive user        1039
1      heavy user         111
2        low user          23
3     medium user          40
4   very low user           2


vendor37 had around 1039 exclusive users and 111 heavy users. They would be most likely to be affected


-    Is there a measurable revenue change as a result of the closure (on April 1st)
-    How about regionally?

To answer the above, we look at the following

- Avg daily revenue for the last week before the closure
- Avg daily revenue for the last 4 same days of week as April 1st
- Avg daily revenu on 1st Arpil

In [ ]:
end_dt=dt.datetime(2020,4,1)


wk_lst = []

for i in range(1,8):
  d=end_dt - dt.timedelta(days=i)

  wk_lst.append(d)



lst_wk_df=df[df['date'].isin(wk_lst)]



lst_4wkday_lst=[]

for i in range(1,5):
  d2=end_dt - dt.timedelta(days=7*i)
  lst_4wkday_lst.append(d2)

lst_4wkday_df=df[df['date'].isin(lst_4wkday_lst)]

april1st_df=df[df['date']==end_dt]

avg_lst_wk=lst_wk_df.groupby(['date'])['revenue'].sum().reset_index()
avg_lst_wk_rev=avg_lst_wk['revenue'].mean().round(2)

avg_lst_4wkday=lst_4wkday_df.groupby(['date'])['revenue'].sum().reset_index()
avg_lst_4wkday_rev=avg_lst_4wkday['revenue'].mean().round(2)

april1st_rev=april1st_df['revenue'].sum().round(2)


print('The avg daily rev of the last week before closure was :'+ str(avg_lst_wk_rev))
print('\n')

print('The avg daily rev of the last 4 same weekdays before closure was :'+ str(avg_lst_4wkday_rev))
print('\n')

print('The avg rev on 1st april was :'+ str(april1st_rev))


###Let's investigate the same metrics for vendor37

df_v37=df[df['vendor']=='vendor37'].sort_values(['date'],ascending=True)

v37_lst_wk_df=df_v37[df_v37['date'].isin(wk_lst)]

v37_lst_4wkday_df=df_v37[df_v37['date'].isin(lst_4wkday_lst)]


v37_avg_lst_wk=v37_lst_wk_df.groupby(['date'])['revenue'].sum().reset_index()
v37_avg_lst_wk_rev=v37_avg_lst_wk['revenue'].mean().round(2)

v37_avg_lst_4wkday=v37_lst_4wkday_df.groupby(['date'])['revenue'].sum().reset_index()
v37_avg_lst_4wkday_rev=v37_avg_lst_4wkday['revenue'].mean().round(2)



print('\n')
print('vendor37 avg daily rev of the last week before closure was :'+ str(v37_avg_lst_wk_rev))
print('\n')

print('vendor37 daily rev of the last 4 same weekdays before closure was :'+ str(v37_avg_lst_4wkday_rev))
print('\n')

print('Although there was a drop in revenue on April 1st, only a small portion of that could be attributed to the closure of vendor37')




The avg daily rev of the last week before closure was :470546.07


The avg daily rev of the last 4 same weekdays before closure was :479657.12


The avg rev on 1st april was :454782.31


vendor37 avg daily rev of the last week before closure was :1857.03


vendor37 daily rev of the last 4 same weekdays before closure was :1359.94


Although there was a drop in revenue on April 1st, only a small portion of that could be attributed to the closure of vendor37


In [ ]:
###Looking at the same by region

avg_lst_wk=lst_wk_df.groupby(['date','country_code'])['revenue'].sum().reset_index()
reg_lst_week=avg_lst_wk.groupby(['country_code'])['revenue'].mean().reset_index().rename(columns={'revenue':'lst_week_rev_avg'})

avg_lst_4wkday=lst_4wkday_df.groupby(['date','country_code'])['revenue'].sum().reset_index()
reg_avg_lst_4wkday=avg_lst_4wkday.groupby(['country_code'])['revenue'].mean().reset_index().rename(columns={'revenue':'lst_4wkday_rev_avg'})


reg_april1st_rev=april1st_df.groupby(['country_code'])['revenue'].sum().round(2).reset_index().rename(columns={'revenue':'april1st_rev'})

reg_april1st_rev=pd.merge(reg_april1st_rev,reg_avg_lst_4wkday,on=['country_code'],how='outer')
reg_april1st_rev=pd.merge(reg_april1st_rev,reg_lst_week,on=['country_code'],how='outer')

reg_april1st_rev=reg_april1st_rev.replace(np.nan,0)
reg_april1st_rev['drop_frm_wkday_avg']=reg_april1st_rev['april1st_rev']-reg_april1st_rev['lst_4wkday_rev_avg']
reg_april1st_rev['drop_frm_lst_week_avg']=reg_april1st_rev['april1st_rev']-reg_april1st_rev['lst_week_rev_avg']

###Let's investigate the same metrics for vendor37
df_v37=df[df['vendor']=='vendor37'].sort_values(['date'],ascending=True)

v37_lst_wk_df=df_v37[df_v37['date'].isin(wk_lst)]

v37_lst_4wkday_df=df_v37[df_v37['date'].isin(lst_4wkday_lst)]


v37_avg_lst_wk=v37_lst_wk_df.groupby(['date','country_code'])['revenue'].sum().reset_index()
v37_avg_lst_wk_rev=v37_avg_lst_wk.groupby(['country_code'])['revenue'].mean().round(2).reset_index().rename(columns={'revenue':'v37_lst_week_rev_avg'})

v37_avg_lst_4wkday=v37_lst_4wkday_df.groupby(['date','country_code'])['revenue'].sum().reset_index()
v37_avg_lst_4wkday_rev=v37_avg_lst_4wkday.groupby(['country_code'])['revenue'].mean().round(2).reset_index().rename(columns={'revenue':'v37_lst_4wkday_rev_avg'})

reg_april1st_rev=pd.merge(reg_april1st_rev,v37_avg_lst_wk_rev,on=['country_code'],how='left')
reg_april1st_rev=pd.merge(reg_april1st_rev,v37_avg_lst_4wkday_rev,on=['country_code'],how='left')

reg_april1st_rev=reg_april1st_rev.replace(np.nan,0)
reg_april1st_rev['v37_share_lst_week']=reg_april1st_rev['v37_lst_week_rev_avg']/reg_april1st_rev['lst_week_rev_avg']

reg_april1st_rev['v37_share_lst_4wkday']=reg_april1st_rev['v37_lst_4wkday_rev_avg']/reg_april1st_rev['lst_4wkday_rev_avg']


biggest_drops_frm_lst_week=reg_april1st_rev[['country_code','april1st_rev','lst_week_rev_avg','drop_frm_lst_week_avg','v37_lst_week_rev_avg']]
biggest_drops_frm_lst_week['rnk']=biggest_drops_frm_lst_week['drop_frm_lst_week_avg'].rank(ascending=True)
biggest_drops_frm_lst_week.sort_values(['rnk'],ascending=[True],inplace=True)

t20_biggest_drops_frm_lst_week=biggest_drops_frm_lst_week[biggest_drops_frm_lst_week['rnk']<20]

print(t20_biggest_drops_frm_lst_week[['country_code','april1st_rev','drop_frm_lst_week_avg','v37_lst_week_rev_avg']])
print('\n')
print('\n')
biggest_drops_frm_lst_4wkday=reg_april1st_rev[['country_code','april1st_rev','lst_4wkday_rev_avg','drop_frm_wkday_avg','v37_lst_4wkday_rev_avg']]
biggest_drops_frm_lst_4wkday['rnk']=biggest_drops_frm_lst_4wkday['drop_frm_wkday_avg'].rank(ascending=True)
biggest_drops_frm_lst_4wkday.sort_values(['rnk'],ascending=[True],inplace=True)

t20_biggest_drops_frm_lst_4wkdy=biggest_drops_frm_lst_4wkday[biggest_drops_frm_lst_4wkday['rnk']<20]
print(t20_biggest_drops_frm_lst_4wkdy[['country_code','april1st_rev','drop_frm_wkday_avg','v37_lst_4wkday_rev_avg']])
print('\n')
print('Vendor37 closure had a very small impact on the drop in revenue.')

print('Countries which had highest revenue drop, had no or very little revenue from vendor37.')
print('\n')
print('country45 &country66 actually did did not lose as much rev as vendor37 had brought in on avg in the last week before closure')

   country_code  april1st_rev  drop_frm_lst_week_avg  v37_lst_week_rev_avg
41     country2     40,960.55    -9,358.676414338566                   0.0
44   country207     19,187.32   -3,262.2933769922856                   0.0
6     country11     27,921.56   -1,631.2214796908556                   0.0
78    country71      2,933.51   -1,058.3450938614283                   0.0
5    country109      39,853.0     -940.5270347592887                   0.0
39   country185      2,459.72     -853.3846743962863                   0.0
11   country124        264.43      -777.892276234857                   0.0
68    country52    223,358.53      -569.356177951704                   0.0
83    country85        580.75     -547.4781560121428                131.01
65    country45      2,055.96     -484.3250220389996                503.56
42    country20      2,875.39     -481.6850803707148                   0.0
76    country66      1,390.92    -434.99645268828544                 707.5
67    country51      1,84

In [ ]:
###Let's explore this a little further. Let's look at which vendors took a huge dip in spending on April 1st


lst_4wkday_df=df[df['date'].isin(lst_4wkday_lst)]
lst_wk_df=df[df['date'].isin(wk_lst)]
april1st_df=df[df['date']==end_dt]

avg_lst_wk=lst_wk_df.groupby(['date','vendor'])['revenue'].sum().reset_index()
avg_lst_wk_rev=avg_lst_wk.groupby(['vendor'])['revenue'].mean().round(2).reset_index().rename(columns={'revenue':'rev_lst_wk_avg'})

avg_lst_4wkday=lst_4wkday_df.groupby(['date','vendor'])['revenue'].sum().reset_index()
avg_lst_4wkday_rev=avg_lst_4wkday.groupby(['vendor'])['revenue'].mean().round(2).reset_index().rename(columns={'revenue':'rev_lst_4wkd_avg'})

april1st_rev=april1st_df.groupby(['vendor'])['revenue'].sum().round(2).reset_index().rename(columns={'revenue':'rev_1stapr'})

comp_df=pd.merge(avg_lst_wk_rev,avg_lst_4wkday_rev,on=['vendor'],how='outer')

comp_df2=pd.merge(comp_df,april1st_rev,on=['vendor'],how='outer')

comp_df2['drop_frm_lst_wk_avg']=(comp_df2['rev_1stapr']-comp_df2['rev_lst_wk_avg']).round(2)
comp_df2['drop_frm_lst_4wkd_avg']=(comp_df2['rev_1stapr']-comp_df2['rev_lst_4wkd_avg']).round(2)
print(comp_df2[['vendor','drop_frm_lst_wk_avg','drop_frm_lst_4wkd_avg']])
print('\nvendor11 and vendor26 had the biggest deviations on april 1st')

     vendor  drop_frm_lst_wk_avg  drop_frm_lst_4wkd_avg
0  vendor11             9,286.67             -12,690.35
1  vendor26           -22,369.83              -7,674.47
2  vendor29               110.87                   6.63
3  vendor33                87.54                -936.55
4  vendor36                217.2                -104.98
5  vendor37                  NaN                    NaN
6  vendor42            -1,239.18              -2,115.15

vendor11 and vendor26 had the biggest deviations on april 1st


How does the spending of vendor37 users after closure compare to the spending of the users of other vendors?

In [ ]:
###We had previously isloated vendor37 users and calssified them. We will use that dataset once again here v37_user_type

df['period']='pre_closure'
df.loc[(df['month']>3),'period']='post_closure'

##Isolation of non vendor37 users
v37_user_lst=list(set(v37_user_type['account'].to_list()))

df['v37_user']='N'

df.loc[(df['account'].isin(v37_user_lst)),'v37_user']='Y'
###Calculation of avg spend per user pre and post closure

m1=df.groupby(['period','v37_user','account'])['revenue'].sum().reset_index().rename(columns={'revenue':'period_spend'})
m2=df.groupby(['period','v37_user','account'])['sku'].count().reset_index().rename(columns={'sku':'period_tranactions'})

m3=pd.merge(m1,m2,on=['period','account','v37_user'],how='inner')

m4=pd.merge(m3,v37_user_type[['account','user_type']],on=['account'],how='left')

m4.loc[(m4['user_type'].isnull()),'user_type']='non_user'

m4['transaction_aov']=m4['period_spend']/m4['period_tranactions']

m5=m4.groupby(['period','user_type'])['transaction_aov'].mean().reset_index().rename(columns={'transaction_aov':'avg_spend'})
m6=m5.pivot(index=['user_type'],columns=['period'],values=['avg_spend']).reset_index()

m6.columns=[x[1] if x else x[0] for x in m6.columns.to_list()]

m6['post_closure']=m6['post_closure'].round(2)
m6['pre_closure']=m6['pre_closure'].round(2)

print(m6)


print('There has been a decline overall apart from the very low users but the number of accounts is extremely small in that case.\nEven users who never transacted with vendor37 have seen average spending declines')

                   post_closure  pre_closure
0  exclusive user         57.39       106.17
1      heavy user         56.12        79.59
2        low user         68.44        83.08
3     medium user          59.4        83.39
4        non_user         85.01        93.02
5   very low user         87.04        41.05
There has been a decline overall apart from the very low users but the number of accounts is extremely small in that case.
Even users who never transacted with vendor37 have seen average spending declines


Is there any indication that vendor37 users have migrated to new vendors?

In [ ]:
v37_user_df=df[(df['v37_user']=='Y')]

v37_user_df=pd.merge(v37_user_df,v37_user_type[['account','user_type']],on=['account'],how='inner')

accnt_period=v37_user_df.groupby(['period','user_type'])['account'].nunique().reset_index()

accnt_period_pivot=accnt_period.pivot(index=['user_type'],columns=['period'],values=['account']).reset_index()                    
###Most users haven't come back and made transactions post the closure
accnt_period_pivot

accnt_period_pivot.columns=[x[1] if x else x[0] for x in accnt_period_pivot.columns.to_list()]
accnt_period_pivot['perc_not_returned']=((1-(accnt_period_pivot['post_closure']/accnt_period_pivot['pre_closure']))*100).round(2)
print(accnt_period_pivot)
#df.to_csv('working_data.csv',index=False)
print('The exclusive users have almost eroded post closure.\nThe heavy users have also not returned in a large majority')

s1=accnt_period_pivot['pre_closure'].sum()
s2=accnt_period_pivot['post_closure'].sum()
s3=((1-s2/s1)*100).round(2)

print('\ntotal % erosion of users has been '+ str(s3) +'%')

                   post_closure  pre_closure  perc_not_returned
0  exclusive user            58         1039              94.42
1      heavy user            21          111              81.08
2        low user             9           23              60.87
3     medium user             9           40               77.5
4   very low user             1            2               50.0
The exclusive users have almost eroded post closure.
The heavy users have also not returned in a large majority

total % erosion of users has been 91.93%
